In [ ]:
#imports 
import csv
import re
import pandas as pd

In [ ]:
#read in original davidson data
davidson_og=pd.read_csv("davidson_labeled_data_copy.csv")

### Character Cleanup

Apply custom clean_tweets function to clean tweets of @tags and emoticon ASCII codes.  

In [ ]:
#present a good example for before and after cleaning
print(davidson_og['tweet'][1556])

In [ ]:
def clean_tweets(tweet): 
    tweet = re.sub('@[^ ]+ ', '', tweet) #remove user @ tags
    tweet = re.sub('&[^;]+;', '', tweet) #remove ascii codes for emojis
    tweet = re.sub('http://[^ ]+','', tweet) #remove http:// hyperlinks
    tweet = re.sub('https://[^ ]+','', tweet) #remove https:// hyperlinks
    tweet = tweet.replace('=', '', 1) #remove '=' at the beginning of some messages
    tweet = tweet.replace('\\n', ' ') #trim out \\n
    
    return tweet

In [ ]:
#apply clean_tweets custom function
davidson_clean = davidson_og.copy()
davidson_clean['tweet'] = davidson_clean['tweet'].apply(clean_tweets)

In [ ]:
#present cleaned tweet example from above
print(davidson_clean['tweet'][1556])

In [ ]:
davidson_clean = davidson_clean.loc[davidson_clean['tweet'] != ''] #remove any blank tweets
davidson_clean.shape

In [ ]:
davidson_clean.to_csv("davidson_labeled_data_replaced.csv", index=False) #create cleaned csv file
davidson_clean.head(3)

## Ambiguous Split

Class 0 = Hate Speech, Class 1 = Offensive Language, Class 2 = Neither

In [ ]:
davidson_replaced = pd.read_csv('davidson_labeled_data_replaced.csv') #read cleaned tweets
keep_col = ['count', 'hate_speech', 'offensive_language', 'neither', 'class', 'tweet'] #created for abbreviation

In [ ]:
davidson_hate = davidson_replaced.loc[(davidson_replaced['offensive_language'] == 0) & (davidson_replaced['neither'] == 0) \
                                    & (davidson_replaced['class'] == 0)] #select tweets that were deemed universally hateful
davidson_hate_tweets = davidson_hate[keep_col]
davidson_hate_tweets.head(3)

In [ ]:
davidson_hate_tweets.to_csv('davidson_hate_only.csv', index=True, header=True)

In [ ]:
davidson_hate_mix = davidson_replaced.loc[(davidson_replaced['count'] != davidson_replaced['hate_speech']) \
                                             & (davidson_replaced['class'] == 0)] #select tweets that had a majority but not unanimous hate vote
davidson_hate_mix_tweets = davidson_hate_mix[keep_col]
davidson_hate_mix_tweets.to_csv("davidson_hate_mix.csv", index=True, header=True)

In [ ]:
davidson_clean = davidson_replaced.loc[(davidson_replaced['offensive_language'] == 0) & (davidson_replaced['hate_speech'] == 0) \
                                    & (davidson_replaced['class'] == 2)] #select tweets that were deemed universally clean content
davidson_clean_tweets = davidson_clean[keep_col]
davidson_clean_tweets.to_csv('davidson_clean_only.csv', index=True, header=True)